## Άσκηση 1

Ευάγγελος Τζώρτζης ΑΜ 3088

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.sparse as sp_sparse
import scipy.spatial.distance as sp_dist

import matplotlib.pyplot as plt

import sklearn as sk
import sklearn.datasets as sk_data
import sklearn.metrics as metrics
from sklearn import preprocessing
import sklearn.cluster as sk_cluster

import sklearn.model_selection as model_selection
from sklearn.model_selection import KFold

from sklearn.utils import shuffle

import sklearn.feature_extraction.text as sk_text

import sklearn.linear_model as linear_model
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

import scipy.cluster.hierarchy as hr

import time
import seaborn as sns

import json

import multiprocessing

%matplotlib inline

###  Load Data

In [2]:
data_file = open("archive/yelp_academic_dataset_business.json", encoding="utf8")
data = []
for line in data_file:
    data.append(json.loads(line))
business_df = pd.DataFrame(data)
data_file.close()

In [3]:
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0..."
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",None
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': '{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'..."


In [4]:
boston_businesses = business_df[business_df.city.isin(['boston','Boston'])]

In [5]:
boston_businesses.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
26,hcRxdDg7DYryCxCoI8ySQA,Longwood Galleria,340-350 Longwood Ave,Boston,MA,02215,42.338544,-71.106842,2.5,24,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Restaurants, Shopping, Shopping Centers","{'Monday': '6:30-22:0', 'Tuesday': '6:30-22:0'..."
29,jGennaZUr2MsJyRhijNBfA,Legal Sea Foods,1 Harborside Dr,Boston,MA,02128,42.363442,-71.025781,3.5,856,1,"{'NoiseLevel': 'u'average'', 'BikeParking': 'F...","Sandwiches, Food, Restaurants, Breakfast & Bru...","{'Monday': '6:0-21:0', 'Tuesday': '6:0-21:0', ..."
35,iPD8BBvea6YldQZPHzVrSQ,Espresso Minute,334 Mass Ave,Boston,MA,02115,42.342673,-71.084239,4.5,7,0,"{'NoiseLevel': ''quiet'', 'GoodForKids': 'True...","Creperies, Restaurants, Food, Coffee & Tea, Br...","{'Tuesday': '8:0-20:0', 'Wednesday': '8:0-20:0..."
89,WuoNSCjFBH4OQTSNGc4l6w,Toscana European Day Spa & Salon,231 Newbury St,Boston,MA,02116,42.350004,-71.081931,4.0,138,1,"{'ByAppointmentOnly': 'False', 'BusinessParkin...","Beauty & Spas, Hair Salons, Active Life, Day Spas","{'Tuesday': '10:0-18:0', 'Wednesday': '10:0-20..."
154,zHMsnpMPanzRx-A6AwGSdw,Ecco Beauty Hair Salon,"2 Boylston St, Ste 101",Boston,MA,02116,42.352135,-71.063102,3.5,42,1,"{'BusinessParking': '{'garage': True, 'street'...","Day Spas, Tanning, Hair Stylists, Massage, Spr...","{'Monday': '10:0-20:0', 'Wednesday': '11:0-18:..."


In [6]:
# TAKES SOME TIME AND A LOT OF MEMORY !!!

b = time.time()

data_file = open("archive/yelp_academic_dataset_review.json", encoding="utf8")
data = []
for line in data_file:
    data.append(json.loads(line))
review_df = pd.DataFrame(data)
data_file.close()

print('Time taken to load into memory:',(time.time()-b))

In [7]:
#review_df.head()

In [8]:
boston_reviews = review_df.loc[review_df.business_id.isin(boston_businesses.business_id)].copy()

In [9]:
boston_reviews['date'] = pd.to_datetime(boston_reviews['date'], errors='coerce')

In [10]:
#boston_reviews

In [11]:
boston_2020_reviews = boston_reviews.loc[boston_reviews['date'].dt.year == 2020].copy()

In [12]:
#boston_2020_reviews

In [17]:
# save final dataframe to file as json so i dont have to load the whole review dataset each time
boston_2020_reviews.to_json(r'boston_2020_reviews_json.json')

In [18]:
boston_2020_reviews_loaded = pd.read_json('boston_2020_reviews_json.json')

In [19]:
boston_2020_reviews_loaded

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
589524,AueHLY8Z4k4CYYHohIsdFg,USEM1lAYDJIwkGVxkSfdVw,UAtX7xmIfdd1W2Pebf6NWg,3,2,0,1,Chill dive bar that was around the corner from...,2020-01-01 19:44:49
589596,UPcBOJL8cghGY2XlR8ss8w,mKH2Q_dPuvExcl6X1FVsvg,jEUDnzLiocHQfifjE7PaoA,5,0,0,0,Definitely a worthy place to stay. The room ...,2020-02-02 13:04:22
589640,-PafLXDjsxA1G3FBEuK--w,ai4Ck-LcuUzfmUaJA5Y3lQ,MjpH-uP90jTUp_KqAOiBJg,2,0,0,0,Once upon Stella was a great restaurant. It wa...,2020-03-07 02:37:49
589648,54Uti6naeFZHbI1nEHF7ew,ORTpbuUmG44UU7oG4QBciA,BOzdu1Rv8ootQpnSBdhrng,5,0,0,0,The spicy calamari is not to be missed! As are...,2020-03-10 01:36:04
589681,0W0N9fReAbPUGC8wDoSHzg,ypEr5IlM8wtZGmPirxXCWA,WQtLtE9GfCsRBPvJGuI_AA,5,0,0,0,It provided great peace of mind to know that w...,2020-05-03 15:34:37
...,...,...,...,...,...,...,...,...,...
8635198,XuGbNPm9PJzn6fosFPCPZg,miUgeCGfYSfy23y5Xg4P5w,eDtkBv6E-JErrLe1td5fvQ,4,0,0,1,This is a small museum with interesting exhibi...,2020-08-05 22:54:22
8635242,6sh_thIAsGzRMDEYL49pfQ,9m-fWJi-fyIBfusfglntsA,3UyAItqUjqod-pnMm0YUmg,1,0,0,0,Poor mannered staff... discriminatory. Every t...,2020-08-05 11:38:48
8635283,1Qp1svmXqb7qSn59HQYIkQ,MqSYy8F_zyeku_T8Na58jg,eDtkBv6E-JErrLe1td5fvQ,3,1,0,0,Honestly kind of a hit or miss...the location ...,2020-01-11 22:20:23
8635346,SqbIik9cIWXaFAmPJ8GBbg,59Gy8CdWmJQXFdmJ3ftC6Q,y2w6rFaO0XEiG5mFfOsiFA,1,1,0,0,AWFUL SERVICE. I called 3 HOURS AHEAD and they...,2020-11-29 01:16:33


### Step 1

In [20]:
# shuffle dataframe
boston_2020_reviews_loaded_shuffled = shuffle(boston_2020_reviews_loaded).copy()
# reset index
boston_2020_reviews_loaded_shuffled.reset_index(inplace=True,drop=True)

In [21]:
boston_2020_reviews_loaded_shuffled

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,9LHw_dkgg-adSGU64Gb0sw,aY-k6JWViY7dALkWdXDsaw,cg8mfYBkdhaPrvEVblm7gQ,1,0,0,0,Horrible!!!!!! I don't care if it's covid or n...,2020-11-27 21:44:44
1,vXhf8BIkPMpnlbSY12Ig7A,xtWusFZFSUoMu-nbuYEwDA,13NuB2aD-tn3b2ZbFsrEag,1,1,0,0,I currently live here and have for the past ye...,2020-01-16 20:12:41
2,twbR0LVFbfDm4hKWVxTbOg,fH1exFvaMB76LO_8s9Q7Rg,LCFZDAaLRpmaidikraTWiw,4,0,0,0,"We started off with the biscuits and butter, w...",2020-01-05 16:34:57
3,o_Vqsx2oyAh3oGwtdnHLOw,ETt7xV3ZFGbk5mynRE7wNw,7qEwmzw4ZN3Ja4Lu458_dA,3,2,0,1,My bf graduated from BU in '14 and during our ...,2020-09-24 23:35:02
4,BlExbTwehgICTzktDdcFCg,TUg-MHoW4k2CMeJwmJWsoA,bC_zfsWM5adXVsx4TAtuZQ,4,0,0,0,Strawberries and French champagne crepe is so ...,2020-01-03 16:19:29
...,...,...,...,...,...,...,...,...,...
32022,A30J-OFesDVrZKXXZagM1g,szNRP1PnxbNxnS1v9wen8Q,DZwndM7SUHCFAkH-TNdhrw,1,0,0,0,The food here is trash. No flavor what so ever...,2020-07-18 04:36:51
32023,gY4f6qIVugUB2J7cnJBQRQ,edWlY2HHDeXMGji_o7Uogg,Iwecaf9sqVNLGD5zI7V-KQ,1,0,0,0,"Dropped off pants to be hemmed on a Tuesday, t...",2020-10-08 18:40:53
32024,DRLCnps91-94BeXWZxOzlQ,Spqv-TES61uVEVEb1c48kQ,QSHnlS5PEIjtx04S3VmvFw,5,0,0,0,"Monica's Mercato you are simply awesome, I had...",2020-12-26 01:07:45
32025,NAHB9A5IIsD_NdPZQCIR2w,IGdwm0hj6uaaoP7xrg0m7A,l4YpEuvIm_RqNXleuDIthw,5,0,0,0,This restaurant is excellent. They are taking ...,2020-12-12 08:26:59


In [22]:
def class_maker(stars):
    if stars >= 4:
        return 1
    return 0

In [23]:
# make positive/ negative classes from stars if stars>=4: positive review (1) , else if stars < 4: negative (0)
boston_2020_reviews_loaded_shuffled['positive_review_class'] = boston_2020_reviews_loaded_shuffled.apply(lambda row: class_maker(row['stars']),axis=1)

In [24]:
boston_2020_reviews_loaded_shuffled

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,positive_review_class
0,9LHw_dkgg-adSGU64Gb0sw,aY-k6JWViY7dALkWdXDsaw,cg8mfYBkdhaPrvEVblm7gQ,1,0,0,0,Horrible!!!!!! I don't care if it's covid or n...,2020-11-27 21:44:44,0
1,vXhf8BIkPMpnlbSY12Ig7A,xtWusFZFSUoMu-nbuYEwDA,13NuB2aD-tn3b2ZbFsrEag,1,1,0,0,I currently live here and have for the past ye...,2020-01-16 20:12:41,0
2,twbR0LVFbfDm4hKWVxTbOg,fH1exFvaMB76LO_8s9Q7Rg,LCFZDAaLRpmaidikraTWiw,4,0,0,0,"We started off with the biscuits and butter, w...",2020-01-05 16:34:57,1
3,o_Vqsx2oyAh3oGwtdnHLOw,ETt7xV3ZFGbk5mynRE7wNw,7qEwmzw4ZN3Ja4Lu458_dA,3,2,0,1,My bf graduated from BU in '14 and during our ...,2020-09-24 23:35:02,0
4,BlExbTwehgICTzktDdcFCg,TUg-MHoW4k2CMeJwmJWsoA,bC_zfsWM5adXVsx4TAtuZQ,4,0,0,0,Strawberries and French champagne crepe is so ...,2020-01-03 16:19:29,1
...,...,...,...,...,...,...,...,...,...,...
32022,A30J-OFesDVrZKXXZagM1g,szNRP1PnxbNxnS1v9wen8Q,DZwndM7SUHCFAkH-TNdhrw,1,0,0,0,The food here is trash. No flavor what so ever...,2020-07-18 04:36:51,0
32023,gY4f6qIVugUB2J7cnJBQRQ,edWlY2HHDeXMGji_o7Uogg,Iwecaf9sqVNLGD5zI7V-KQ,1,0,0,0,"Dropped off pants to be hemmed on a Tuesday, t...",2020-10-08 18:40:53,0
32024,DRLCnps91-94BeXWZxOzlQ,Spqv-TES61uVEVEb1c48kQ,QSHnlS5PEIjtx04S3VmvFw,5,0,0,0,"Monica's Mercato you are simply awesome, I had...",2020-12-26 01:07:45,1
32025,NAHB9A5IIsD_NdPZQCIR2w,IGdwm0hj6uaaoP7xrg0m7A,l4YpEuvIm_RqNXleuDIthw,5,0,0,0,This restaurant is excellent. They are taking ...,2020-12-12 08:26:59,1


In [77]:
# construct classifier models

# Logistic regression:
lr_clf = linear_model.LogisticRegression(solver='lbfgs')

# SVM:
svm_clf = svm.SVC()

# K-nn
knn = KNeighborsClassifier(n_neighbors=10) # todo check different number of neighbors

In [26]:
# vectorizer for the reviews
boston_2020_reviews_loaded_shuffled['text'][-10:]

32017    Haunt my life.  Poor customer service!   I got...
32018    Great food! Inside and outside seating. Excell...
32019    November 1st, 2020\n\nPlaced a simple pickup o...
32020    Love!! Order one of everything on the menu and...
32021    Cheap date night is a MUST! Monday and Tuesday...
32022    The food here is trash. No flavor what so ever...
32023    Dropped off pants to be hemmed on a Tuesday, t...
32024    Monica's Mercato you are simply awesome, I had...
32025    This restaurant is excellent. They are taking ...
32026    Super trendy spot but food fell short for me. ...
Name: text, dtype: object

In [27]:
vectorizer = sk_text.TfidfVectorizer(stop_words='english',
                             max_features = 1000,
                             min_df=4, max_df=0.8)

In [28]:
train_temp = vectorizer.fit_transform(boston_2020_reviews_loaded_shuffled['text'])


In [29]:
train_temp.shape

(32027, 1000)

In [41]:
#vectorizer.get_feature_names()  # old sklearn method, for updated sklearn library use get_feature_names_out()

In [31]:
lr_clf.fit(train_temp[:25000],boston_2020_reviews_loaded_shuffled['positive_review_class'][:25000])

LogisticRegression()

In [32]:
test_temp = vectorizer.transform(boston_2020_reviews_loaded_shuffled['text'][-7027:])

In [33]:
lr_clf.score(test_temp,boston_2020_reviews_loaded_shuffled['positive_review_class'][-7027:])

0.902091931122812

In [34]:
metrics.recall_score(boston_2020_reviews_loaded_shuffled['positive_review_class'][-7027:],lr_clf.predict(test_temp))

0.9466472908449243

In [35]:
# test k fold
kf  = KFold(shuffle=False)
for train_index, test_index in kf.split(boston_2020_reviews_loaded_shuffled['text']):
    print("TRAIN:", train_index, "TEST:", test_index)
boston_2020_reviews_loaded_shuffled.iloc[test_index]

TRAIN: [ 6406  6407  6408 ... 32024 32025 32026] TEST: [   0    1    2 ... 6403 6404 6405]
TRAIN: [    0     1     2 ... 32024 32025 32026] TEST: [ 6406  6407  6408 ... 12809 12810 12811]
TRAIN: [    0     1     2 ... 32024 32025 32026] TEST: [12812 12813 12814 ... 19214 19215 19216]
TRAIN: [    0     1     2 ... 32024 32025 32026] TEST: [19217 19218 19219 ... 25619 25620 25621]
TRAIN: [    0     1     2 ... 25619 25620 25621] TEST: [25622 25623 25624 ... 32024 32025 32026]


,review_id,user_id,business_id,stars,useful,funny,cool,text,date,positive_review_class
25622,cQ-udcQ9m5rqSMQ2CWNYtA,rJAVLXInHzThFbJGbqOPDQ,2SPPzZLYOieBSjsXRuUJAg,5,0,0,0,We used Supreme Home Cleaning for a few years ...,2020-01-20 20:26:02,1
25623,ytXUeQCkfsjRnMOx_7ElAg,vsVaIycCnCnDUMm6AMFwRQ,QzAZ1tB0NaJ6VIzcO_SPQQ,1,0,0,0,Worst Hotel in Boston! Will not stay again. $5...,2020-09-08 19:38:08,0
25624,94nAT-QEwna5BDVKemr2OA,In3YGO6yxdLjX0bAdsj89g,6yPo1VyadJozt8KBTOhPdQ,5,0,0,0,Authentic Spicy Scezhuan food in China Town Bo...,2020-01-03 00:55:21,1
25625,KENpWCxbbeUsXsbtS_eu4A,YBTT-hKmcMzDEyPjN6hzKQ,lzJ2QFiScrLJkGsjzXjXxQ,5,1,1,1,"Best lobster roll in Boston. Hands down, back ...",2020-11-17 20:31:52,1
25626,89dFQDyvzX7fIczIogyX_Q,jX28At-fTE5BRQgyFlxv1w,QF_XZtqJqyyaUjNop7uS_A,2,2,0,0,We had heard so many great things about this r...,2020-07-08 14:18:05,0
...,...,...,...,...,...,...,...,...,...,...
32022,A30J-OFesDVrZKXXZagM1g,szNRP1PnxbNxnS1v9wen8Q,DZwndM7SUHCFAkH-TNdhrw,1,0,0,0,The food here is trash. No flavor what so ever...,2020-07-18 04:36:51,0
32023,gY4f6qIVugUB2J7cnJBQRQ,edWlY2HHDeXMGji_o7Uogg,Iwecaf9sqVNLGD5zI7V-KQ,1,0,0,0,"Dropped off pants to be hemmed on a Tuesday, t...",2020-10-08 18:40:53,0
32024,DRLCnps91-94BeXWZxOzlQ,Spqv-TES61uVEVEb1c48kQ,QSHnlS5PEIjtx04S3VmvFw,5,0,0,0,"Monica's Mercato you are simply awesome, I had...",2020-12-26 01:07:45,1
32025,NAHB9A5IIsD_NdPZQCIR2w,IGdwm0hj6uaaoP7xrg0m7A,l4YpEuvIm_RqNXleuDIthw,5,0,0,0,This restaurant is excellent. They are taking ...,2020-12-12 08:26:59,1


In [61]:
importances = pd.DataFrame(data={
    'Attribute': vectorizer.get_feature_names(),
    'Importance': lr_clf.coef_[0]
})

In [62]:
print(importances.sort_values(by='Importance',ascending=False)[:20]['Attribute'].to_numpy())

['amazing' 'delivery' 'greeted' 'perfection' 'delivered' 'best' 'lovely'
 'hit' 'excited' 'fast' 'thank' 'wonderful' 'fee' 'person' 'low' 'indoor'
 'haven' 'boston' 'provided' 'fries']


In [63]:
importances.sort_values(by='Importance',ascending=False)[-20:]

,Attribute,Importance
926,unless,-3.023722
47,asked,-3.135894
61,awful,-3.451728
987,worse,-3.652142
594,old,-3.653780
533,mention,-3.675235
665,pork,-3.715143
90,bland,-3.848092
923,understand,-4.044459
56,average,-4.067567


In [71]:
def runLogisticRegressionKfold():
    print('Logistic Regression:')
    b = time.time()
    stats = []
    conf = []
    kf  = KFold(shuffle=False)
    for train_index, test_index in kf.split(boston_2020_reviews_loaded_shuffled['text']):
        #print("TRAIN:", train_index, "TEST:", test_index) # show the splits of the indices
        boston_2020_reviews_loaded_shuffled.iloc[test_index]
    
        vectorizer = sk_text.TfidfVectorizer(stop_words='english',max_features = 1000, min_df=4, max_df=0.8)
    
        train_temp = vectorizer.fit_transform(boston_2020_reviews_loaded_shuffled.iloc[train_index]['text'])
        
        lr_clf.fit(train_temp,boston_2020_reviews_loaded_shuffled.iloc[train_index]['positive_review_class'])
        
        test_temp = vectorizer.transform(boston_2020_reviews_loaded_shuffled['text'][test_index])
        
        #print(lr_clf.score(test_temp,boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'])) # = accuracy score
        
        conf.append(metrics.confusion_matrix(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],lr_clf.predict(test_temp)))
        
        stats.append([])
        stats[-1].append(metrics.accuracy_score(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],lr_clf.predict(test_temp)))
        stats[-1].append(metrics.precision_score(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],lr_clf.predict(test_temp)))
        stats[-1].append(metrics.recall_score(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],lr_clf.predict(test_temp)))
        stats[-1].append(metrics.f1_score(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],lr_clf.predict(test_temp)))

        
    
    final_conf = []
    final_conf.append([])
    final_conf.append([])
    final_conf[0].append((conf[0][0][0]+conf[1][0][0]+conf[2][0][0]+conf[3][0][0])/4)
    final_conf[0].append((conf[0][0][1]+conf[1][0][1]+conf[2][0][1]+conf[3][0][1])/4)
    final_conf[1].append((conf[0][1][0]+conf[1][1][0]+conf[2][1][0]+conf[3][1][0])/4)
    final_conf[1].append((conf[0][1][1]+conf[1][1][1]+conf[2][1][1]+conf[3][1][1])/4)

    print('\nTotal time taken: ',time.time()-b,'\n')
    
    print(' Confusion Matrix: \n',np.asarray(final_conf))
    
    final_stats = np.array(stats)
    final_stats = np.mean(final_stats, axis=0)
    print (' Accuracy: ',final_stats[0],'\n',
           'Precision: ',final_stats[1],'\n',
           'Recall: ',final_stats[2],'\n',
           'F1_score: ',final_stats[3],'\n',)

    # 20 words with the biggest positive weights and 20 words with the smallest negative weights for the last fold regression
    importances = pd.DataFrame(data={
    'Attribute': vectorizer.get_feature_names(),
    'Importance': lr_clf.coef_[0]
    })
    
    most_important_words = importances.sort_values(by='Importance',ascending=False)[:20]['Attribute'].to_numpy() # important for positive reviews
    least_important_words = importances.sort_values(by='Importance',ascending=False)[-20:]['Attribute'].to_numpy() # important for negative reviews
    
    print('Most important words for positive reviews (20 most positive weights in the model):\n',most_important_words,'\n')
    print('Most important words for negative reviews (20 most negative weights in the model):\n',least_important_words)    

In [72]:
runLogisticRegressionKfold()

Logistic Regression:

Total time taken:  12.797102212905884 

 Confusion Matrix: 
 [[1623.    394.75]
 [ 225.25 4162.5 ]]
 Accuracy:  0.9036439003929523 
 Precision:  0.9142353332047316 
 Recall:  0.9484396397238731 
 F1_score:  0.9310214987514461 

Most important words for positive reviews (20 most positive weights in the model):
 ['amazing' 'delicious' 'great' 'perfect' 'definitely' 'best' 'love'
 'highly' 'excellent' 'fantastic' 'thank' 'wonderful' 'favorite'
 'perfectly' 'loved' 'incredible' 'happy' 'boston' 'professional'
 'friendly'] 

Most important words for negative reviews (20 most negative weights in the model):
 ['unprofessional' 'asked' 'awful' 'worse' 'okay' 'mediocre' 'poor' 'bland'
 'unfortunately' 'average' 'told' 'overpriced' 'money' 'disappointed' 'ok'
 'terrible' 'horrible' 'disappointing' 'rude' 'worst']


In [108]:
def runSVM_Kfold():
    print('SVM:')
    b = time.time()
    stats = []
    conf = []
    kf  = KFold(shuffle=False)
    for train_index, test_index in kf.split(boston_2020_reviews_loaded_shuffled['text']):
        #print("TRAIN:", train_index, "TEST:", test_index) # show the splits of the indices
        boston_2020_reviews_loaded_shuffled.iloc[test_index]
    
        vectorizer = sk_text.TfidfVectorizer(stop_words='english',max_features = 1000, min_df=4, max_df=0.8)
    
        train_temp = vectorizer.fit_transform(boston_2020_reviews_loaded_shuffled.iloc[train_index]['text'])
        
        svm_clf.fit(train_temp,boston_2020_reviews_loaded_shuffled.iloc[train_index]['positive_review_class'])
        
        test_temp = vectorizer.transform(boston_2020_reviews_loaded_shuffled['text'][test_index])
        
        #print(svm_clf.score(test_temp,boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'])) # = accuracy score
        
        conf.append(metrics.confusion_matrix(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],svm_clf.predict(test_temp)))
        
        stats.append([])
        stats[-1].append(metrics.accuracy_score(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],svm_clf.predict(test_temp)))
        stats[-1].append(metrics.precision_score(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],svm_clf.predict(test_temp)))
        stats[-1].append(metrics.recall_score(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],svm_clf.predict(test_temp)))
        stats[-1].append(metrics.f1_score(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],svm_clf.predict(test_temp)))

        print('Time passed: ',time.time()-b)
    
    final_conf = []
    final_conf.append([])
    final_conf.append([])
    final_conf[0].append((conf[0][0][0]+conf[1][0][0]+conf[2][0][0]+conf[3][0][0])/4)
    final_conf[0].append((conf[0][0][1]+conf[1][0][1]+conf[2][0][1]+conf[3][0][1])/4)
    final_conf[1].append((conf[0][1][0]+conf[1][1][0]+conf[2][1][0]+conf[3][1][0])/4)
    final_conf[1].append((conf[0][1][1]+conf[1][1][1]+conf[2][1][1]+conf[3][1][1])/4)

    print('\nTotal time taken: ',time.time()-b,'\n')
    
    print(' Confusion Matrix: \n',np.asarray(final_conf))
    
    final_stats = np.array(stats)
    final_stats = np.mean(final_stats, axis=0)
    print (' Accuracy: ',final_stats[0],'\n',
           'Precision: ',final_stats[1],'\n',
           'Recall: ',final_stats[2],'\n',
           'F1_score: ',final_stats[3],'\n',)


In [109]:
runSVM_Kfold()

SVM:
Time passed:  261.55549788475037
Time passed:  523.7385699748993
Time passed:  784.0958776473999
Time passed:  1041.6307022571564
Time passed:  1301.2376399040222

Total time taken:  1301.2376399040222 

 Confusion Matrix: 
 [[1626.25  391.5 ]
 [ 219.75 4168.  ]]
 Accuracy:  0.9053924465329757 
 Precision:  0.9150218330067104 
 Recall:  0.9502653842383422 
 F1_score:  0.932310123529934 



In [82]:
def runKnn_Kfold():
    print('Knn:')
    b = time.time()
    stats = []
    conf = []
    kf  = KFold(shuffle=False)
    for train_index, test_index in kf.split(boston_2020_reviews_loaded_shuffled['text']):
        #print("TRAIN:", train_index, "TEST:", test_index) # show the splits of the indices
        boston_2020_reviews_loaded_shuffled.iloc[test_index]
    
        vectorizer = sk_text.TfidfVectorizer(stop_words='english',max_features = 1000, min_df=4, max_df=0.8)
    
        train_temp = vectorizer.fit_transform(boston_2020_reviews_loaded_shuffled.iloc[train_index]['text'])
        
        knn.fit(train_temp,boston_2020_reviews_loaded_shuffled.iloc[train_index]['positive_review_class'])
        
        test_temp = vectorizer.transform(boston_2020_reviews_loaded_shuffled['text'][test_index])
        
        #print(knn.score(test_temp,boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'])) # = accuracy score
        
        conf.append(metrics.confusion_matrix(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],knn.predict(test_temp)))
        
        stats.append([])
        stats[-1].append(metrics.accuracy_score(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],knn.predict(test_temp)))
        stats[-1].append(metrics.precision_score(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],knn.predict(test_temp)))
        stats[-1].append(metrics.recall_score(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],knn.predict(test_temp)))
        stats[-1].append(metrics.f1_score(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],knn.predict(test_temp)))

        print(' Confusion Matrix:\n',metrics.confusion_matrix(boston_2020_reviews_loaded_shuffled.iloc[test_index]['positive_review_class'],knn.predict(test_temp)))
        print('Time passed: ',time.time()-b)
    
    final_conf = []
    final_conf.append([])
    final_conf.append([])
    final_conf[0].append((conf[0][0][0]+conf[1][0][0]+conf[2][0][0]+conf[3][0][0])/4)
    final_conf[0].append((conf[0][0][1]+conf[1][0][1]+conf[2][0][1]+conf[3][0][1])/4)
    final_conf[1].append((conf[0][1][0]+conf[1][1][0]+conf[2][1][0]+conf[3][1][0])/4)
    final_conf[1].append((conf[0][1][1]+conf[1][1][1]+conf[2][1][1]+conf[3][1][1])/4)

    print('\nTotal time taken: ',time.time()-b,'\n')
    
    print(' Confusion Matrix: \n',np.asarray(final_conf))
    
    final_stats = np.array(stats)
    final_stats = np.mean(final_stats, axis=0)
    print (' Accuracy: ',final_stats[0],'\n',
           'Precision: ',final_stats[1],'\n',
           'Recall: ',final_stats[2],'\n',
           'F1_score: ',final_stats[3],'\n',)


In [83]:
runKnn_Kfold()

Knn:
 Confusion Matrix:
 [[  68 1972]
 [  32 4334]]
Time passed:  46.513500690460205
 Confusion Matrix:
 [[  68 1862]
 [  41 4435]]
Time passed:  94.50642371177673
 Confusion Matrix:
 [[  44 2032]
 [  23 4306]]
Time passed:  142.33830499649048
 Confusion Matrix:
 [[  66 1959]
 [  33 4347]]
Time passed:  188.97463607788086
 Confusion Matrix:
 [[  65 1933]
 [  20 4387]]
Time passed:  236.8953652381897

Total time taken:  236.8953652381897 

 Confusion Matrix: 
 [[  61.5  1956.25]
 [  32.25 4355.5 ]]
 Accuracy:  0.6906669757055923 
 Precision:  0.6908937546011635 
 Recall:  0.9932250371939115 
 F1_score:  0.814889453016366 



#### Συμπεράσματα:


Τα μοντέλα έτρεξαν με 1000 features.

Η λογιστική παλινδρόμηση έχει αρκετά υψηλές όλες τις μετρικές, καθώς και αρκετά καλό confusion matrix, με τα περισσότερα παραδείγματα να έχουν σωστή ταξινόμηση σε θετική ή αρνητική βαθμολογία.

Όπως είναι αναμενόμενο οι 20 πιο σημαντικές λέξεις για θετική ταξινόμηση είναι λέξεις που εκφράζουν κάτι θετικό: 'amazing' 'delicious' 'great' 'perfect' 'definitely' 'best' 'love'
 'highly' 'excellent' 'fantastic' 'thank' 'wonderful' 'favorite'
 'perfectly' 'loved' 'incredible' 'happy' 'boston' 'professional'
 'friendly'
 
και οι λέξεις που που δίνουν τις πιο αρνητικές κριτικές είναι: 'unprofessional' 'asked' 'awful' 'worse' 'okay' 'mediocre' 'poor' 'bland'
 'unfortunately' 'average' 'told' 'overpriced' 'money' 'disappointed' 'ok'
 'terrible' 'horrible' 'disappointing' 'rude' 'worst'
 
To SVM παίρνει την περισσότερη ώρα να τρέξει (~ 25 λεπτά) και δίνει παρόμοιες μετρικές με το logistic regression.
 
To Κnn με 10 γείτονες είναι γρήγορο αλλά οι μετρικές που δίνει είναι χαμηλότερες από τα άλλα δύο μοντέλα με αρκετά χαμηλά accuracy,precision και λίγο χαμηλότερο f1 score. Ωστόσω τα recall είναι μεγαλύτερο. Το confusion matrix φαίνεται διαφορετικό από τα προηγούμενα μοντέλα.

### Step 2

In [94]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 
import string as string

import gensim 
from gensim.models import Word2Vec
import gensim.downloader as api

In [95]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Βαγγέλης\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [96]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Βαγγέλης\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [97]:
english_stop_words = set(stopwords.words('english'))

In [98]:
# tokenize words
X_total_nltk = []
y_total_nltk = []
for x,y in zip(boston_2020_reviews_loaded_shuffled.text,boston_2020_reviews_loaded_shuffled.positive_review_class):
    wt = word_tokenize(x.lower())
    doc = [w for w in wt if (w not in english_stop_words) and (w not in string.punctuation)]
    if len(doc) == 0: continue
    X_total_nltk.append(doc)
    y_total_nltk.append(y)


In [99]:
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

C:\Users\Βαγγέλης/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz


In [100]:
path = 'C:\\Users\\Βαγγέλης/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz' # custom path
g_model = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)

In [101]:
# Transform the data
X_total_gmodel = []
for x in X_total_nltk:
    vx = np.zeros(300)
    length = 0
    for w in x: 
        if w in g_model:
            length += 1
            vx += g_model[w]
    if length != 0: vx /= length
    X_total_gmodel.append(vx)

In [102]:
lr_clf2 = linear_model.LogisticRegression(solver='sag') # lbfgs doesn't converge

In [103]:
scores = model_selection.cross_validate(lr_clf2,
                                          #svm_clf,
                                          #knn,
                                          X_total_gmodel,
                                          y_total_nltk,
                                          scoring=['accuracy','precision','recall','f1'],
                                          cv=5)
#print (scores)
print (' Accuracy:  ',scores['test_accuracy'].mean(),'\n',
       'Precision: ',scores['test_precision'].mean(),'\n',
       'Recall:    ',scores['test_recall'].mean(),'\n',
       'F1 score:  ',scores['test_f1'].mean())

 Accuracy:   0.8971182266430062 
 Precision:  0.9099486082807399 
 Recall:     0.9433008048836438 
 F1 score:   0.9263230066573968


In [104]:
def calc_models():
    print('Linear Regression:\n')
    b = time.time()
    scores = model_selection.cross_validate(lr_clf2,
                                          #svm_clf,
                                          #knn,
                                          X_total_gmodel,
                                          y_total_nltk,
                                          scoring=['accuracy','precision','recall','f1'],
                                          cv=5)
    print('Time taken: ',time.time()-b)
    #print (scores)
    print (' Accuracy:  ',scores['test_accuracy'].mean(),'\n',
           'Precision: ',scores['test_precision'].mean(),'\n',
           'Recall:    ',scores['test_recall'].mean(),'\n',
           'F1 score:  ',scores['test_f1'].mean())
    
    print('\nSVM:\n')
    b = time.time()
    scores = model_selection.cross_validate(#lr_clf2,
                                          svm_clf,
                                          #knn,
                                          X_total_gmodel,
                                          y_total_nltk,
                                          scoring=['accuracy','precision','recall','f1'],
                                          cv=5)
    print('Time taken: ',time.time()-b)
    #print (scores)
    print (' Accuracy:  ',scores['test_accuracy'].mean(),'\n',
           'Precision: ',scores['test_precision'].mean(),'\n',
           'Recall:    ',scores['test_recall'].mean(),'\n',
           'F1 score:  ',scores['test_f1'].mean())
    
    print('\nK-nn:\n')
    b = time.time()
    scores = model_selection.cross_validate(#lr_clf2,
                                          #svm_clf,
                                          knn,
                                          X_total_gmodel,
                                          y_total_nltk,
                                          scoring=['accuracy','precision','recall','f1'],
                                          cv=5)
    print('Time taken: ',time.time()-b)
    #print (scores)
    print (' Accuracy:  ',scores['test_accuracy'].mean(),'\n',
           'Precision: ',scores['test_precision'].mean(),'\n',
           'Recall:    ',scores['test_recall'].mean(),'\n',
           'F1 score:  ',scores['test_f1'].mean())

In [105]:
calc_models()

Linear Regression:

Time taken:  10.504814386367798
 Accuracy:   0.8971182266430062 
 Precision:  0.9099486082807399 
 Recall:     0.9433008048836438 
 F1 score:   0.9263230066573968

SVM:

Time taken:  376.54586601257324
 Accuracy:   0.90355025282455 
 Precision:  0.9155316074160696 
 Recall:     0.9466707236485956 
 F1 score:   0.9308371724754544

K-nn:

Time taken:  18.76282238960266
 Accuracy:   0.8354200967428321 
 Precision:  0.8804575199864221 
 Recall:     0.8793601770304302 
 F1 score:   0.8798997962046677


#### Συμπεράσματα:

Το logistic regression με τα word embeddings παρουσιάζει σχεδόν ίδια αποτελέσματα με το αντίστοιχο tf-idf.

Το SVM ομοίως έχει πανομοιότυπες μετρικές και στις δύο εκδοχές.

Το K-nn μοντέλο έχει τη μεγαλύτερη βελτίωση με χρήση embeddings και οι μετρικές αυξάνονται αρκετά εκτός από το recall.